In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import os


In [48]:
df = pd.read_csv(r"C:\Users\dev\Desktop\road_condition_project\dataset\road_condition_dataset.csv.csv")
df.head()


,timestamp,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,label
0,0,0.745071,-0.207396,0.971533,-0.050143,0.764114,-0.788581,Kankar Road
1,1,2.284545,-0.351230,-0.351205,0.403237,-0.424206,-0.634298,Kankar Road
2,2,2.368819,1.151152,-0.704212,-0.085624,-0.828194,-0.442919,Kankar Road
3,3,0.813840,-0.695127,-0.698595,-0.958302,1.571780,0.028211,Kankar Road
4,4,0.362943,-2.869920,-2.587377,-0.559780,0.171184,-0.089862,Kankar Road


In [50]:
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

# Normalize sensor values
features = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

print("Label mapping:", dict(zip(le.classes_, le.transform(le.classes_))))


Label mapping: {'Bitumen Road': 0, 'Concrete Road': 1, 'Kankar Road': 2, 'Multiple Speed Breakers': 3, 'Single Speed Breaker': 4}


In [52]:
X = df[features].values
y = df['label_encoded'].values

# Make sure total rows are divisible by 150
num_samples = len(X) // 150
X = X[:num_samples * 150].reshape(num_samples, 150, 6)
y = y[:num_samples * 150].reshape(num_samples, 150)

# Use the mode (most common) label for each 3-second chunk
from scipy.stats import mode
y = mode(y, axis=1)[0].flatten()

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (50, 150, 6)
y shape: (50,)


In [60]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [98]:
# STEP 6: Build LSTM model
model = Sequential([
    Input(shape=(150, 6)),  # or (150, 8) if you added extra features
    LSTM(128, return_sequences=True),
    Dropout(0.4),
    LSTM(64),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_10 (LSTM)                  │ (None, 150, 128)       │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 150, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 123,013 (480.52 KB)

 Trainable params: 123,013 (480.52 KB)

 Non-trainable params: 0 (0.00 B)

In [104]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# === STEP 7: Set Callbacks ===
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model.keras", save_best_only=True)
]


In [110]:
# STEP 7: Train model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=callbacks
)


Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step - accuracy: 0.8917 - loss: 0.2171 - val_accuracy: 0.7000 - val_loss: 0.3514
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 308ms/step - accuracy: 0.9187 - loss: 0.1836 - val_accuracy: 0.8000 - val_loss: 0.3202
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step - accuracy: 0.9563 - loss: 0.1309 - val_accuracy: 0.8000 - val_loss: 0.3201
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.8854 - loss: 0.1867 - val_accuracy: 0.7000 - val_loss: 0.4076
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - accuracy: 0.9187 - loss: 0.1621 - val_accuracy: 0.7000 - val_loss: 0.6323
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.9187 - loss: 0.1757 - val_accuracy: 0.7000 - val_loss: 0.3374
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step - accuracy: 0.9187 - loss: 0.1531 - val_accuracy: 0.8000 - val_loss: 0.2439
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - accuracy: 0.9458 - loss: 0.1337 - val_accuracy: 0.9000 - v

In [112]:
# STEP 8: Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")

# STEP 9: Save model
os.makedirs("model", exist_ok=True)
model.save("model/road_condition_model.h5")
print("Model saved successfully!")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.9000 - loss: 0.1070


Test Accuracy: 90.00%
Model saved successfully!


In [3]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the trained model
model = load_model(r"C:\Users\dev\Desktop\road_condition_project\notebooks\model\road_condition_model.h5")

# Simulate a 3-second sample (150 time steps × 6 features)
sample_input = np.random.rand(1, 150, 6)  # Replace with real or preprocessed data

# Predict road condition
pred = model.predict(sample_input)
predicted_label = np.argmax(pred, axis=1)

# Map encoded label to actual class name
labels = ['Bitumen Road', 'Concrete Road', 'Kankar Road', 'Multiple Speed Breakers', 'Single Speed Breaker']
print("Predicted Road Condition:", labels[predicted_label[0]])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step
Predicted Road Condition: Kankar Road
